![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/competitions/miad2024-12-prediccion-precio-vehiculos).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importación librerías
import pandas as pd
import numpy as np

In [ ]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)

In [ ]:
# Visualización datos de entrenamiento
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [ ]:
# Visualización datos de test
dataTesting.head()

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand


In [ ]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
np.random.seed(42)
y_pred = pd.DataFrame(np.random.rand(dataTesting.shape[0]) * 75000 + 5000, index=dataTesting.index, columns=['Price'])

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred.to_csv('test_submission.csv', index_label='ID')
y_pred.head()

,Price
ID,
0,33090.508914
1,76303.572981
2,59899.545636
3,49899.386315
4,16701.398033


## Librerías

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


## 0. Preprocesamiento de Datos

Los datos de entrenamiento se dividen en datos de entrenamiento y validación. Si decidieron preprocesar los datos (estandarizar, normalizar, imputar valores, etc), estos son correctamente preprocesados al ajustar sobre los datos de entrenamiento (.fit_transform()) y al transformar los datos del set de validación (.transform()). (10 puntos)

In [4]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)

In [ ]:
# Visualización datos de entrenamiento
print(dataTraining.head
())

   Price  Year  Mileage State       Make      Model
0  34995  2017     9913    FL       Jeep   Wrangler
1  37895  2015    20578    OH  Chevrolet   Tahoe4WD
2  18430  2012    83716    TX        BMW      X5AWD
3  24681  2014    28729    OH   Cadillac  SRXLuxury
4  26998  2013    64032    CO       Jeep   Wrangler


In [ ]:
# Visualización datos de test
print(dataTesting.head())

    Year  Mileage State    Make        Model
ID                                          
0   2014    31909    MD  Nissan    MuranoAWD
1   2017     5362    FL    Jeep     Wrangler
2   2014    50300    OH    Ford  FlexLimited
3   2004   132160    WA     BMW            5
4   2015    25226    MA    Jeep        Grand


In [5]:
dataTraining['State'] = dataTraining['State'].str.strip()
# Convertir variables categóricas en variables dummy
dataTraining = pd.get_dummies(dataTraining, columns=['State', 'Make', 'Model'])

# Visualizar los primeros registros del conjunto de datos preprocesado
print(dataTraining.head())

   Price  Year  Mileage  State_AK  State_AL  State_AR  State_AZ  State_CA  \
0  34995  2017     9913     False     False     False     False     False   
1  37895  2015    20578     False     False     False     False     False   
2  18430  2012    83716     False     False     False     False     False   
3  24681  2014    28729     False     False     False     False     False   
4  26998  2013    64032     False     False     False     False     False   

   State_CO  State_CT  ...  Model_Yaris4dr  Model_YarisBase  Model_YarisLE  \
0     False     False  ...           False            False          False   
1     False     False  ...           False            False          False   
2     False     False  ...           False            False          False   
3     False     False  ...           False            False          False   
4      True     False  ...           False            False          False   

   Model_Yukon  Model_Yukon2WD  Model_Yukon4WD  Model_Yukon4dr  Mode

In [7]:
# Definir las características (X) y la variable objetivo (y)
X = dataTraining.drop('Price', axis=1)
y = dataTraining['Price']

# Dividir los datos en conjuntos de entrenamiento y prueba
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
xgboost = XGBRegressor(random_state=42)

# Definir hiperparámetros a buscar para cada modelo
param_grid_xgb = {'n_estimators': [100, 200, 300],
                  'learning_rate': [0.05, 0.1, 0.2]}

# Realizar búsqueda de hiperparámetros para cada modelo
grid_search_xgb = GridSearchCV(xgboost, param_grid_xgb, cv=5)

# Entrenar modelos con la mejor combinación de hiperparámetros
grid_search_xgb.fit(XTrain, yTrain)

# Obtener los mejores modelos
best_xgb = grid_search_xgb.best_estimator_

# Imprimir mensaje de finalización
print("Entrenamiento completado.")

KeyboardInterrupt: 

In [ ]:
# Evaluar el rendimiento en el conjunto de prueba
y_pred_xgb = best_xgb.predict(XTest)
mse_xgb = mean_squared_error(yTest, y_pred_xgb)

# Imprimir las métricas de rendimiento
print("MSE Decision Tree:", mse_dt)
print("MSE Random Forest:", mse_rf)
print("MSE Gradient Boosting:", mse_gb)
print("MSE XGBoost:", mse_xgb)


#------------------FINAL

In [ ]:
# Inicializar el modelo con los hiperparámetros ajustados
xgb_model = XGBRegressor(
    booster="gbtree",
    colsample_bytree=1,
    eta=0.3,
    gamma=1,
    max_depth=9,
    max_leaves=0,
    n_estimators=540,
    objective="reg:linear",
    reg_alpha=2.291666666666667,
    reg_lambda=1.7708333333333335,
    tree_method="auto",
    subsample=0.7,
    random_state=42
)

# Entrenar el modelo con los datos de entrenamiento
xgb_model.fit(XTrain, yTrain)

# Imprimir mensaje de finalización
print("Entrenamiento completado.")


Entrenamiento completado.


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Realizar predicciones en el conjunto de prueba
y_pred = xgb_model.predict(XTest)

# Calcular métricas de rendimiento
mse = mean_squared_error(yTest, y_pred)
mae = mean_absolute_error(yTest, y_pred)
r2 = r2_score(yTest, y_pred)
rmse = np.sqrt(mse)

# Imprimir las métricas de rendimiento
print("Métricas de rendimiento del modelo:")
print("Error cuadrático medio (MSE):", mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)
print("Error absoluto medio (MAE):", mae)
print("Coeficiente de determinación (R^2):", r2)


Métricas de rendimiento del modelo:
Error cuadrático medio (MSE): 11997074.91404786
Raíz del error cuadrático medio (RMSE): 3463.679389615595
Error absoluto medio (MAE): 2171.490903515625
Coeficiente de determinación (R^2): 0.8963045725274584


In [ ]:
# Convertir variables categóricas en variables numéricas
dataTesting_encoded = pd.get_dummies(dataTesting, columns=['State', 'Make', 'Model'])

# Asegurarse de que las columnas del conjunto de prueba coincidan con las del conjunto de entrenamiento
missing_cols = set(XTrain.columns) - set(dataTesting_encoded.columns)
for col in missing_cols:
    dataTesting_encoded[col] = 0
dataTesting_encoded = dataTesting_encoded[XTrain.columns]

# Realizar predicciones en el conjunto de prueba preprocesado
predictions = xgb_model.predict(dataTesting_encoded)
print(dataTesting_encoded)

       Year  Mileage  State_ AK  State_ AL  State_ AR  State_ AZ  State_ CA  \
ID                                                                            
0      2014    31909      False      False      False      False      False   
1      2017     5362      False      False      False      False      False   
2      2014    50300      False      False      False      False      False   
3      2004   132160      False      False      False      False      False   
4      2015    25226      False      False      False      False      False   
...     ...      ...        ...        ...        ...        ...        ...   
99995  2015    82719      False      False      False      False      False   
99996  2015    19711      False      False      False      False      False   
99997  2016    48049      False      False      False      False       True   
99998  2017    15306      False      False      False      False      False   
99999  2013    40393      False      False      Fals

In [ ]:
# Crear un DataFrame con las predicciones para subir a la plataforma
predicciones_XGB = pd.DataFrame({'ID': dataTesting_encoded.index, 'Price': predictions})
predicciones_XGB.to_csv('predicciones_XGB_7.csv', index=False)
print(predicciones_XGB)

          ID         Price
0          0  21179.744141
1          1  37660.210938
2          2  23639.357422
3          3   7963.722168
4          4  31302.402344
...      ...           ...
99995  99995  19815.648438
99996  99996  38392.316406
99997  99997  23235.943359
99998  99998  17244.031250
99999  99999  18472.423828

[100000 rows x 2 columns]


#-------------- FINAL

In [ ]:
# Crear un DataFrame con las predicciones para subir a la plataforma
predicciones_XGB = pd.DataFrame({'Id': datos_test['Id'], 'y': XGB_predictions})
predicciones_XGB.to_csv('predicciones_XGB_50.csv', index=False)
print(predicciones_XGB)

# Exploracion de Hiperparametros

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Definir rangos para cada hiperparámetro
param_grid = {
    'booster': ['gbtree'],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'eta': [0.1, 0.2, 0.3],
    'gamma': [0, 1, 5],
    'max_depth': [4, 6, 8],
    'max_leaves': [0],  # Considera ajustar este rango si es necesario
    'n_estimators': [50, 100, 200],
    'objective': ['reg:linear'],
    'reg_alpha': [0, 1, 2],
    'reg_lambda': [0, 1, 2],
    'subsample': [0.6, 0.7, 0.8],
    'tree_method': ['auto'],
    'random_state': [42]
}

# Inicializar el modelo
xgb_model = XGBRegressor()

# Realizar la búsqueda de cuadrícula
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Entrenar el modelo con los datos de entrenamiento
grid_search.fit(XTrain, yTrain)

# Obtener el mejor modelo
best_xgb_model = grid_search.best_estimator_

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Imprimir mensaje de finalización
print("Entrenamiento completado.")


# Con TruncateSVD

In [ ]:
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Crear una instancia de TruncatedSVD con el número de componentes especificado
svd = TruncatedSVD(n_components=4, random_state=42)

# Aplicar TruncatedSVD a los datos de entrenamiento y prueba
XTrain_svd = svd.fit_transform(XTrain)
XTest_svd = svd.transform(XTest)

# Inicializar y entrenar el modelo XGBoostRegressor
xgb_model_truncate = XGBRegressor(
    booster="gbtree",
    colsample_bytree=1,
    eta=0.3,
    gamma=1,
    max_depth=6,
    max_leaves=0,
    n_estimators=100,
    objective="reg:linear",
    reg_alpha=2.291666666666667,
    reg_lambda=1.7708333333333335,
    subsample=0.7,
    tree_method="auto",
    random_state=42
)
xgb_model_truncate.fit(XTrain_svd, yTrain)

# Realizar predicciones en el conjunto de prueba
y_pred_truncate = xgb_model_truncate.predict(XTest_svd)

# Calcular y imprimir el error cuadrático medio (MSE)
mse = mean_squared_error(yTest, y_pred_truncate)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)


MSE: 63986450.577424645
RMSE: 7999.153116263286


# Con StandardScaler

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Definir el pipeline con el preprocesamiento y el modelo XGBoostRegressor
pipeline = Pipeline([
    ('xgb_model', XGBRegressor(
        booster="gbtree",
        colsample_bytree=0.5,
        eta=0.5,
        gamma=1,
        max_depth=7,
        max_leaves=3,
        n_estimators=100,
        objective="reg:linear",
        reg_alpha=0,
        reg_lambda=0.625,
        subsample=0.9,
        tree_method="auto",
        random_state=42
    ))
])

# Entrenar el modelo utilizando el pipeline
pipeline.fit(XTrain, yTrain)

# Realizar predicciones en el conjunto de prueba
y_pred_scaler = pipeline.predict(XTest)

# Calcular y imprimir el error cuadrático medio (MSE)
mse = mean_squared_error(yTest, y_pred_scaler)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)


MSE: 32139380.142154627
RMSE: 5669.160444206411


In [8]:
# Inicializar modelos
decision_tree = DecisionTreeRegressor(random_state=42)
random_forest = RandomForestRegressor(random_state=42)
gradient_boosting = GradientBoostingRegressor(random_state=42)
xgboost = XGBRegressor(random_state=42)

# Entrenar modelos con la mejor combinación de hiperparámetros
decision_tree.fit(XTrain, yTrain)
random_forest.fit(XTrain, yTrain)
gradient_boosting.fit(XTrain, yTrain)
xgboost.fit(XTrain, yTrain)

# Imprimir mensaje de finalización
print("Entrenamiento completado.")

Entrenamiento completado.


In [9]:
# Evaluar el rendimiento en el conjunto de prueba
y_pred_dt = decision_tree.predict(XTest)
mse_dt = mean_squared_error(yTest, y_pred_dt)

y_pred_rf = random_forest.predict(XTest)
mse_rf = mean_squared_error(yTest, y_pred_rf)

y_pred_gb = gradient_boosting.predict(XTest)
mse_gb = mean_squared_error(yTest, y_pred_gb)

y_pred_xgb = xgboost.predict(XTest)
mse_xgb = mean_squared_error(yTest, y_pred_xgb)

# Imprimir las métricas de rendimiento
print("MSE Decision Tree:", mse_dt)
print("MSE Random Forest:", mse_rf)
print("MSE Gradient Boosting:", mse_gb)
print("MSE XGBoost:", mse_xgb)

MSE Decision Tree: 22432060.730712265
MSE Random Forest: 14099652.58585069
MSE Gradient Boosting: 44904078.2162739
MSE XGBoost: 18135800.237015836
